**Classes e objetos**

In [1]:
class PartyAnimal :
    x = 0
    def _init_(self) :
        print("I am constructed")
    
    def party(self):
        self.x = self.x +1
        print('So far', self.x)

    def _del_(self) :
        print("I am destructed", self.x)

an = PartyAnimal()
an = PartyAnimal()
an.party()
an.party()
an.party()
an = -1
print("an contains", an)

So far 1
So far 2
So far 3
an contains -1


In [3]:
import sqlite3

# Create a database in RAM
db = sqlite3.connect(':memory:')

# Get a cursor object
cursor = db.cursor()
cursor.execute('''
    CREATE TABLE Ages ( 
  name VARCHAR(128), 
  age INTEGER
)''')


cursor.execute('''DELETE FROM Ages''')

# Insert users
cursor.execute('''INSERT INTO Ages (name, age) VALUES ('Kirsten', 37)''')
cursor.execute('''INSERT INTO Ages (name, age) VALUES ('Schekina', 29)''')
cursor.execute('''INSERT INTO Ages (name, age) VALUES ('Ryhs', 36)''')
cursor.execute('''INSERT INTO Ages (name, age) VALUES ('Kadey', 32)''')

#Select user
cursor.execute('''SELECT hex(name || age) AS X FROM Ages ORDER BY X''')

#retrieve the first row
user1 = cursor.fetchone()
#Print the first column retrieved(user's name)
print("The first row in the resulting record set : "+user1[0])

#Commit changes into database
db.commit()
#Close database
db.close()



The first row in the resulting record set : 4B616465793332


In [8]:
import sqlite3

conn = sqlite3.connect('emaildb.sqlite')
cur = conn.cursor()

cur.execute('''
DROP TABLE IF EXISTS Counts''')

cur.execute('''
CREATE TABLE Counts (org TEXT, count INTEGER)''')

fname = input('Enter file name: ')
if ( len(fname) < 1 ) : fname = 'mbox.txt'
fh = open(fname)
for line in fh:
    if not line.startswith('From: ') : continue
    pieces = line.split()
    email = pieces[1]
    parts = email.split('@')
    org = parts[-1]
    cur.execute('SELECT count FROM Counts WHERE org = ? ', (org, ))
    row = cur.fetchone()
    if row is None:
        cur.execute('''INSERT INTO Counts (org, count) 
                VALUES ( ?, 1 )''', ( org, ) )
    else :
        cur.execute('UPDATE Counts SET count=count+1 WHERE org = ?',
            (org, ))
    conn.commit()

sqlstr = 'SELECT org, count FROM Counts ORDER BY count DESC LIMIT 10'

print
print("Counts:")
for row in cur.execute(sqlstr) :
    print (str(row[0]), row[1])

cur.close()

Counts:
iupui.edu 536
umich.edu 491
indiana.edu 178
caret.cam.ac.uk 157
vt.edu 110
uct.ac.za 96
media.berkeley.edu 56
ufp.pt 28
gmail.com 25
et.gatech.edu 17


In [10]:
import xml.etree.ElementTree as ET
import sqlite3
# Use Itunes File, Library, export to get xml file for your own
conn = sqlite3.connect('trackdb.sqlite')
cur = conn.cursor()  # use to send commands

# Make some fresh tables using executescript()
cur.executescript('''
DROP TABLE IF EXISTS Artist;
DROP TABLE IF EXISTS Genre;
DROP TABLE IF EXISTS Album;
DROP TABLE IF EXISTS Track;
CREATE TABLE Artist (
    id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    name    TEXT UNIQUE
);
CREATE TABLE Genre (
    id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    name    TEXT UNIQUE
);
CREATE TABLE Album (
    id  INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    artist_id  INTEGER,
    title   TEXT UNIQUE
);
CREATE TABLE Track (
    id  INTEGER NOT NULL PRIMARY KEY 
        AUTOINCREMENT UNIQUE,
    title TEXT  UNIQUE,
    album_id  INTEGER,
    genre_id  INTEGER,
    len INTEGER, rating INTEGER, count INTEGER
);
''')


fname = input('Enter file name: ')
if ( len(fname) < 1 ) : fname = 'Library.xml'

# <key>Track ID</key><integer>369</integer>
# <key>Name</key><string>Another One Bites The Dust</string>
# <key>Artist</key><string>Queen</string>
def lookup(d, key):
    found = False
    for child in d:
        if found : return child.text
        if child.tag == 'key' and child.text == key :
            found = True
    return None

stuff = ET.parse(fname)  # element tree
all = stuff.findall('dict/dict/dict')  # 3-deep dictionaries
print ('Dict count:'), len(all)
for entry in all:
    if ( lookup(entry, 'Track ID') is None ) : continue
    
    track = lookup(entry, 'TRACK ID')
    name = lookup(entry, 'Name')
    artist = lookup(entry, 'Artist')
    genre = lookup(entry, 'Genre')
    album = lookup(entry, 'Album')


    if name is None or artist is None or genre is None or album is None : 
        continue

    print (track, name, artist, genre, album)

    cur.execute('''INSERT OR IGNORE INTO Artist (name) 
        VALUES ( ? )''', ( artist, ) )
    cur.execute('SELECT id FROM Artist WHERE name = ? ', (artist, ))
    artist_id = cur.fetchone()[0]
    
    
    cur.execute('''INSERT OR IGNORE INTO Genre (name) 
        VALUES ( ? )''', ( genre, ))
    cur.execute('SELECT id FROM Genre WHERE name = ? ', (genre, ))
    genre_id = cur.fetchone()[0]
    

    cur.execute('''INSERT OR IGNORE INTO Album (title, artist_id) 
        VALUES ( ?, ? )''', ( album, artist_id ) )
    cur.execute('SELECT id FROM Album WHERE title = ? ', (album, ))
    album_id = cur.fetchone()[0]
    

    cur.execute('''INSERT OR REPLACE INTO Track 
        (id, title, album_id, genre_id) 
        VALUES ( ?, ?, ?, ? )''', 
        (track, name, album_id, genre_id) )
 
    conn.commit()

Dict count:
None Another One Bites The Dust Queen Rock Greatest Hits
None Asche Zu Asche Rammstein Industrial Herzeleid
None Beauty School Dropout Various Soundtrack Grease
None Black Dog Led Zeppelin Rock IV
None Bring The Boys Back Home Pink Floyd Rock The Wall [Disc 2]
None Circles Bryan Lee Funk Blues Is
None Comfortably Numb Pink Floyd Rock The Wall [Disc 2]
None Crazy Little Thing Called Love Queen Rock Greatest Hits
None Electric Funeral Black Sabbath Metal Paranoid
None Fat Bottomed Girls Queen Rock Greatest Hits
None For Those About To Rock (We Salute You) AC/DC Rock Who Made Who
None Four Sticks Led Zeppelin Rock IV
None Furious Angels Rob Dougan Soundtrack The Matrix Reloaded
None Gelle Bryan Lee Blues/R&B Blues Is
None Going To California Led Zeppelin Rock IV
None Grease Various Soundtrack Grease
None Hand of Doom Black Sabbath Metal Paranoid
None Hells Bells AC/DC Rock Who Made Who
None Hey You Pink Floyd Rock The Wall [Disc 2]
None I Worry Bryan Lee Blues/R&B Blues Is
Non

In [17]:
import json
import sqlite3

conn = sqlite3.connect('rosterdb.sqlite')
cur = conn.cursor()

# Do some setup
cur.executescript('''
DROP TABLE IF EXISTS User;
DROP TABLE IF EXISTS Member;
DROP TABLE IF EXISTS Course;
CREATE TABLE User (
    id     INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    name   TEXT UNIQUE
);
CREATE TABLE Course (
    id     INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT UNIQUE,
    title  TEXT UNIQUE
);
CREATE TABLE Member (
    user_id     INTEGER,
    course_id   INTEGER,
    role        INTEGER,
    PRIMARY KEY (user_id, course_id)
)
''')

fname = input('Enter file name: ')
if len(fname) < 1:
    fname = 'roster_data_sample.json'

# [
#   [ "Charley", "si110", 1 ],
#   [ "Mea", "si110", 0 ],

str_data = open(fname).read()
json_data = json.loads(str_data)

for entry in json_data:

    name = entry[0];
    title = entry[1];
    role = entry[2];

    print((name, title, role))

    cur.execute('''INSERT OR IGNORE INTO User (name)
        VALUES ( ? )''', ( name, ) )
    cur.execute('SELECT id FROM User WHERE name = ? ', (name, ))
    user_id = cur.fetchone()[0]

    cur.execute('''INSERT OR IGNORE INTO Course (title)
        VALUES ( ? )''', ( title, ) )
    cur.execute('SELECT id FROM Course WHERE title = ? ', (title, ))
    course_id = cur.fetchone()[0]

    cur.execute('''INSERT OR REPLACE INTO Member
        (user_id, course_id, role) VALUES ( ?, ?, ? )''',
        ( user_id, course_id, role ) )

    conn.commit()


    
    cur.execute('''SELECT User.name,Course.title, Member.role FROM 
    User JOIN Member JOIN Course 
    ON User.id = Member.user_id AND Member.course_id = Course.id
    ORDER BY User.name DESC, Course.title DESC, Member.role DESC LIMIT 2;''')
    
    #SQL COMMAND 2
    SELECT 'XYZZY' || hex(User.name || Course.title || Member.role ) AS X FROM 
    User JOIN Member JOIN Course 
    ON User.id = Member.user_id AND Member.course_id = Course.id
    ORDER BY X LIMIT 1;  #OUTPUT: XYZZY416172726F6E736933333430
    

SyntaxError: invalid syntax (976425878.py, line 71)